## Setup a classification experiment

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Explore the dataset

In [ ]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

## Train an Explainable Boosting Machine (EBM) and DP-EBM

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier

ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)

from interpret.privacy import DPExplainableBoostingClassifier

dpebm = DPExplainableBoostingClassifier(epsilon=4, delta=1e-6)
dpebm.fit(X_train, y_train)

## Global Explanations: What the model learned overall

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

dpebm_global = dpebm.explain_global(name='DP-EBM')
show(dpebm_global)

## Local Explanations: How an individual prediction was made

In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local)

dpebm_local = dpebm.explain_local(X_test[:5], y_test[:5], name='DP-EBM')
show(dpebm_local)

## Evaluate EBM performance

In [ ]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

dpebm_perf = ROC(dpebm.predict_proba).explain_perf(X_test, y_test, name='DP-EBM')
show(dpebm_perf)

# Model Editing

### Interpretability, Then What? Editing Machine Learning Models to Reflect Human Knowledge and Values

__Zijie J. Wang (Georgia Tech)__; Alex Kale (University of Washington); __Harsha Nori (Microsoft Research)__; Peter Stella (NYU Langone Health); Mark Nunnally (NYU Langone health); Duen Horng Chau (Georgia Institute of Technology); Mihaela Vorvoreanu (Microsoft); Jennifer Wortman Vaughan (Microsoft Research); __Rich Caruana (Microsoft Research)__

__Bold__* Attending KDD 2022

### Applied Data Science Track | Tuesday, August 16, 4:00 PM-6:00 PM, Room 207A (Human & Interfaces) 

In [ ]:
# Edit unwanted learned effects in EBM/DP-EBM with GAM Changer!
import gamchanger as gc

# Load GAM Changer with the model and sample data
gc.visualize(ebm, X_train, y_train)